## Load the Data

In [ ]:
! mkdir data

In [ ]:
! curl --remote-name-all https://www.clarin.si/repository/xmlui/bitstream/handle/11356/1860{/ParlaMint-ES.ana.tgz}

In [ ]:
! mv ParlaMint-ES.ana.tgz data

In [ ]:
! tar -xzf data/ParlaMint-ES.ana.tgz

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

import spacy

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

import re

In [3]:
base_file_path = 'data/ParlaMint-ES.txt'
all_dirs = os.listdir(base_file_path)
all_dirs.remove('00README.txt')

all_dfs = []
all_speech_dfs = []

for dir in all_dirs:
    file_dir = base_file_path + '/' + dir
    files = os.listdir(file_dir)
    for file in files:
        file_path = file_dir + '/' + file
        if '.tsv'  in file_path:
            data = pd.read_csv(file_path, sep='\t')
            if '-en' in file_path:
                data['is_in_english'] = True
            else:
                data['is_in_english'] = False
            all_dfs.append(data)
        elif '.txt' in file_path:
            with open(file_path, 'r') as f:
                # read each line
                lines = f.readlines()
                all_speech_data = {'key':[], 'speech_text':[]}
                for line in lines:
                    # split after first space
                    key = re.split(r'[\t\s]+', line, 1)
                    if len(key) == 2:
                        all_speech_data['key'].append(key[0])
                        all_speech_data['speech_text'].append(key[1])
                    else:
                        print('did not split correctly')
                        print(line)
                speech_df = pd.DataFrame(all_speech_data)
            all_speech_dfs.append(speech_df)


                    


In [4]:
all_data = pd.concat(all_dfs)
all_speech_data = pd.concat(all_speech_dfs)

all_data.head()

,Text_ID,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,...,Speaker_minister,Speaker_party,Speaker_party_name,Party_status,Party_orientation,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,is_in_english
0,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u1,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,-,-,-,-,-,-,-,-,-,False
1,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u2,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,-,-,-,-,-,-,-,-,-,False
2,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u3,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,notMinister,PP,Partido Popular,Opposition,Centro-derecha - Derecha,JaimeDeOlanoVela,"De Olano Vela, Jaime",M,1970,False
3,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u4,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,-,-,-,-,-,-,-,-,-,False
4,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u5,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Minister,PSOE,Partido Socialista Obrero Español,Coalition,Centro-izquierda,NadiaCalviñoSantamaría,"Calviño Santamaría, Nadia",F,1968,False


In [5]:
#all_data = all_data.loc[all_data.Speaker_party.isin(['PP', 'PSOE'])]

In [6]:
all_data = all_data.loc[all_data.Speaker_role.isin(['Diputado/a'])]
print(all_data.shape)

(32551, 24)


In [7]:
all_speech_data.head()

,key,speech_text
0,ParlaMint-ES_2022-09-20-CD220920.u1,"Señorías, se abre la sesión. De conformidad co..."
1,ParlaMint-ES_2022-09-20-CD220920.u2,Comenzamos con el debate de toma en considerac...
2,ParlaMint-ES_2022-09-20-CD220920.u3,"Buenas tardes, señorías. En primer lugar, quie..."
3,ParlaMint-ES_2022-09-20-CD220920.u4,Un segundo. Continúe.\n
4,ParlaMint-ES_2022-09-20-CD220920.u5,"Gracias, presidente. Les decía que, después de..."


In [8]:
all_data = all_data.merge(all_speech_data, left_on='ID', right_on='key')

In [9]:
all_data.head()

,Text_ID,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,...,Speaker_party_name,Party_status,Party_orientation,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,is_in_english,key,speech_text
0,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u3,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Partido Popular,Opposition,Centro-derecha - Derecha,JaimeDeOlanoVela,"De Olano Vela, Jaime",M,1970,False,ParlaMint-ES_2022-06-29-CD220629.u3,"Gracias, presidenta. Señora Calviño, los dos ú..."
1,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u5,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Partido Socialista Obrero Español,Coalition,Centro-izquierda,NadiaCalviñoSantamaría,"Calviño Santamaría, Nadia",F,1968,False,ParlaMint-ES_2022-06-29-CD220629.u5,"Gracias, presidenta. Voy a responder a la preg..."
2,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u7,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Partido Popular,Opposition,Centro-derecha - Derecha,JaimeDeOlanoVela,"De Olano Vela, Jaime",M,1970,False,ParlaMint-ES_2022-06-29-CD220629.u7,"Señora Calviño, menos mal que funcionan sus me..."
3,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u9,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Partido Socialista Obrero Español,Coalition,Centro-izquierda,NadiaCalviñoSantamaría,"Calviño Santamaría, Nadia",F,1968,False,ParlaMint-ES_2022-06-29-CD220629.u9,"Muchas gracias. Ya es mala suerte, señor Olano..."
4,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u12,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Vox,Opposition,Derecha - Extrema derecha,IvánEspinosaDeLosMonterosDeSimón,"Espinosa De Los Monteros De Simón, Iván",M,1971,False,ParlaMint-ES_2022-06-29-CD220629.u12,"Gracias, señora presidenta. Señora vicepreside..."


In [10]:



lemmatizer = WordNetLemmatizer()




# Ensure spaCy uses DataFrame efficiently
def spacy_lemmatize_series(text_series, language='es_core_news_sm'):
    sp = spacy.load(language)
    
    # Disable unnecessary components to speed up processing
    disabled_components = sp.disable_pipes(["ner", "parser"])
    lemmatized_series = text_series.apply(lambda doc: " ".join([token.lemma_ for token in sp(doc)]))
    disabled_components.restore()
    return lemmatized_series

def preprocess_corpus(corpus_series):
    # Remove non-words, handle case, and remove stopwords in one pass
    stop_words = set(stopwords.words('spanish'))
    processed_corpus = corpus_series.apply(lambda doc: " ".join([word.lower() if not re.match('([A-Z]+[a-z]*){2,}', word) else word for word in re.split(r'\W+', doc) if word.lower() not in stop_words]))
    return processed_corpus

def stem_series(corpus_series, stem_type='porter', language='spanish'):
    if stem_type == 'lancaster':
        stemmer = LancasterStemmer()
    elif stem_type == 'porter':
        stemmer = PorterStemmer()
    elif language is not None:
        stemmer = SnowballStemmer(language)
    stemmed_series = corpus_series.apply(lambda doc: " ".join([stemmer.stem(word) for word in doc.split()]))
    return stemmed_series


# Assuming corpus_data is your DataFrame and TXT_EN is the text column
all_data['speech_text_preprocessed'] = spacy_lemmatize_series(all_data['speech_text'])
all_data['speech_text_preprocessed'] = preprocess_corpus(all_data['speech_text_preprocessed'])
#all_data['speech_text_preprocessed'] = stem_series(all_data['speech_text_preprocessed'], stem_type='porter')

all_data

,Text_ID,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,...,Party_status,Party_orientation,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,is_in_english,key,speech_text,speech_text_preprocessed
0,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u3,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Opposition,Centro-derecha - Derecha,JaimeDeOlanoVela,"De Olano Vela, Jaime",M,1970,False,ParlaMint-ES_2022-06-29-CD220629.u3,"Gracias, presidenta. Señora Calviño, los dos ú...",gracias presidenta señora calviño dos último d...
1,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u5,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Coalition,Centro-izquierda,NadiaCalviñoSantamaría,"Calviño Santamaría, Nadia",F,1968,False,ParlaMint-ES_2022-06-29-CD220629.u5,"Gracias, presidenta. Voy a responder a la preg...",gracias presidenta ir responder pregunta tener...
2,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u7,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Opposition,Centro-derecha - Derecha,JaimeDeOlanoVela,"De Olano Vela, Jaime",M,1970,False,ParlaMint-ES_2022-06-29-CD220629.u7,"Señora Calviño, menos mal que funcionan sus me...",señora calviño menos mal funcionar medida infl...
3,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u9,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Coalition,Centro-izquierda,NadiaCalviñoSantamaría,"Calviño Santamaría, Nadia",F,1968,False,ParlaMint-ES_2022-06-29-CD220629.u9,"Muchas gracias. Ya es mala suerte, señor Olano...",gracia ser malo suerte señor olano poner usted...
4,ParlaMint-ES_2022-06-29-CD220629,ParlaMint-ES_2022-06-29-CD220629.u12,Sesión plenaria núm. 191 (2022-06-29),2022-06-29,Cámara Baja,Legislatura XIV,Sesión plenaria núm. 191,-,2022-06-29,-,...,Opposition,Derecha - Extrema derecha,IvánEspinosaDeLosMonterosDeSimón,"Espinosa De Los Monteros De Simón, Iván",M,1971,False,ParlaMint-ES_2022-06-29-CD220629.u12,"Gracias, señora presidenta. Señora vicepreside...",gracias señora presidenta señora vicepresident...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32546,ParlaMint-ES_2016-11-15-CD161115,ParlaMint-ES_2016-11-15-CD161115.u120,Sesión plenaria núm. 14 (2016-11-15),2016-11-15,Cámara Baja,Legislatura XII,Sesión plenaria núm. 14,-,2016-11-15,-,...,Opposition,Centro-izquierda - izquierda,GabrielRufiánRomero,"Rufián Romero, Gabriel",M,1982,False,ParlaMint-ES_2016-11-15-CD161115.u120,"Señora presidenta, por alusiones, quería conte...",señora presidentar alusión querer contestar se...
32547,ParlaMint-ES_2016-11-15-CD161115,ParlaMint-ES_2016-11-15-CD161115.u122,Sesión plenaria núm. 14 (2016-11-15),2016-11-15,Cámara Baja,Legislatura XII,Sesión plenaria núm. 14,-,2016-11-15,-,...,Opposition,Centro-izquierda - izquierda,GabrielRufiánRomero,"Rufián Romero, Gabriel",M,1982,False,ParlaMint-ES_2016-11-15-CD161115.u122,Es la falsedad que ha comentado el portavoz de...,ser falsedad haber comentar portavoz grupo soc...
32548,ParlaMint-ES_2016-11-15-CD161115,ParlaMint-ES_2016-11-15-CD161115.u124,Sesión plenaria núm. 14 (2016-11-15),2016-11-15,Cámara Baja,Legislatura XII,Sesión plenaria núm. 14,-,2016-11-15,-,...,Opposition,Centro-izquierda,MiguelÁngelHerediaDíaz,"Heredia Díaz, Miguel Ángel",M,1966,False,ParlaMint-ES_2016-11-15-CD161115.u124,"Señora presidenta, tengo un vídeo donde el señ...",señora presidenta tener vídeo señor rufián cua...
32549,ParlaMint-ES_2016-11-15-CD161115,ParlaMint-ES_2016-11-15-CD161115.u126,Sesión plenaria núm. 14 (2016-11-15),2016-11-15,Cámara Baja,Legislatura XII,Sesión plenaria núm. 14,-,2016-11-15,-,...,Opposition,Centro-izquierda,MiguelÁngelHerediaDíaz,"Heredia Díaz, Miguel Ánge

In [11]:
# save preprocessed data
all_data.to_csv('all_parliament_data_no_party_removal_diputados_only.csv', index=False)


In [ ]:
# build simple pipeline with tfidf vectorizer and logistic regression

